In [0]:
!pip install tf-nightly --quiet --upgrade

In [2]:
!pip install tensorflow_datasets

# TextVectorization layer examples

In [3]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

tf.__version__

'2.2.0-dev20200305'

## Loading data: IMDB movie review sentiment classification

In [4]:
# Load the IMDB reviews dataset using tfds. This is the raw data.
imdb_reviews = tfds.load('imdb_reviews', in_memory=True)

# The IMDB dataset contains a train split and a test split; we create a separate
# handle for each.
train_raw = imdb_reviews['train']
test_raw = imdb_reviews['test']

# Once we have our handles, we format the datasets in a Keras-fit compatible
# format: a tuple of the form (text_data, label).
def format_dataset(input_data):
  return (input_data['text'], input_data['label'])

train_dataset = train_raw.map(format_dataset)
test_dataset = test_raw.map(format_dataset)

# We also create a dataset with only the textual data in it. This will be used
# to build our vocabulary later on.
text_dataset = train_raw.map(lambda data: data['text'])

In [5]:
# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
# This is one of the places where eager execution shines:
# we can just evaluate these tensors using .numpy()
# instead of needing to evaluate them in a Session/Graph context.
for item in train_raw.take(4):
  print(item['label'].numpy())
  print(item['text'].numpy())

0
b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
0
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development

## Preparing the data

In particular, removing `<br />` tags.

In [0]:
import string
import re

# Having looked at our data above, we see that the raw text contains HTML break
# tags of the form '<br />'. These tags will not be removed by the default
# standardizer (which doesn't strip HTML). Because of this, we will need to 
# create a custom standardization function.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

## Using the TextVectorization layer in an Embedding + Conv1D model

In [0]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Model constants.
max_features = 20000
embedding_dim = 128

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=500)

# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.
vectorize_layer.adapt(text_dataset.batch(64))

# Next, let's build a model.

In [0]:
from tensorflow.keras import layers

# A text input.
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')

# The first layer in our model is the vectorization layer. After this layer,
# we have a tensor of shape (batch_size, max_len) containing vocab indices.
x = vectorize_layer(text_input)

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'. Note that we're using max_features+1 here, since there's an
# OOV token that gets added to the vocabulary in vectorize_layer.
x = layers.Embedding(max_features + 1, embedding_dim)(x)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)
x = layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation='sigmoid', name='predictions')(x)

model = tf.keras.Model(text_input, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(
    loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
batch_size = 32
epochs = 3

# Fit the model using the train and test datasets.
model.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/3
782/782 [==============================] - 168s 214ms/step - loss: 0.4781 - accuracy: 0.7310 - val_loss: 0.2901 - val_accuracy: 0.8788
Epoch 2/3
782/782 [==============================] - 167s 213ms/step - loss: 0.2163 - accuracy: 0.9145 - val_loss: 0.3194 - val_accuracy: 0.8792
Epoch 3/3
782/782 [==============================] - 167s 213ms/step - loss: 0.1115 - accuracy: 0.9615 - val_loss: 0.4127 - val_accuracy: 0.8738


## Using the TextVectorization layer in a bigram TF-IDF densely-connected model

In [0]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Model constants.
max_features = 20000

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    ngrams=2,  # Unigrams and bigrams
    output_mode='tf-idf')
vectorize_layer.adapt(text_dataset.batch(64))

In [0]:
from tensorflow.keras import layers

# A text input.
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')

# The first layer in our model is the vectorization layer. After this layer,
# we have a tensor of shape (batch_size, features) containing TF-IDF features.
x = vectorize_layer(text_input)

# Dense hidden layers
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation='sigmoid', name='predictions')(x)

model = tf.keras.Model(text_input, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(
    loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
batch_size = 32
epochs = 3

# Fit the model using the train and test datasets.
model.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/3
782/782 [==============================] - 485s 621ms/step - loss: 0.6237 - accuracy: 0.7444 - val_loss: 0.2995 - val_accuracy: 0.8824
Epoch 2/3
782/782 [==============================] - 477s 610ms/step - loss: 0.3476 - accuracy: 0.8294 - val_loss: 0.2772 - val_accuracy: 0.8881
Epoch 3/3
782/782 [==============================] - 476s 609ms/step - loss: 0.2942 - accuracy: 0.8523 - val_loss: 0.2825 - val_accuracy: 0.8860
